In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd
from util import *
df = pd.read_csv("movies.csv")
df["profit_margin"] = profit_margin(df["budget"], df["revenue"])
df.head()

In [ ]:
df = multihot_column(df, "-", "genres")
df = multihot_tf_idf(df, "keywords")
# tf idf is being weird for taglines, don't use for now
# df = multihot_tf_idf(df, "tagline")
df.head()

In [ ]:
for i in df.columns:
    print(i)

In [ ]:
for i in df["status"].unique():
    print(i, df["status"][df["status"] == i].shape)

In [ ]:
df["tagline"] = df["tagline"].fillna("_")
df["tagline_count"] = df.apply(lambda row: len(row["tagline"].split(" ")), axis=1)
for i in df[df["tagline_count"] == 1]["tagline"].unique():
    print(i)

In [ ]:
df[df["tagline_count"] > 1]["tagline_count"].hist()

In [ ]:
df[df["vote_average"] != 0]["vote_average"].hist()

In [ ]:
# df[df["vote_count"] != 0]["vote_count"].hist()
df.describe()
print(df.shape)
df[df["vote_count"] == 0].shape[0] / df.shape[0]

In [ ]:
df[df["vote_average"] == 0].shape[0] / df.shape[0]

In [ ]:
df["runtime"].unique()

In [ ]:
df[df["budget"] < 5000000]["budget"]

In [ ]:
credits = set()
for i in df["credits"].dropna():
    # print(i.split('-'))
    for j in i.split("-"):
        # print(j)
        if j not in credits:
            credits.add(j)
len(credits) / df.shape[0]

In [ ]:
df["credits"] = df["credits"].fillna("_")
df["credit_count"] = df.apply(lambda row: len(row["credits"].split("-")), axis=1)
print(sorted(df["credit_count"].unique()))
df[(df["credit_count"] < 100) & (df["credit_count"] > 1)]["credit_count"].hist()

In [ ]:
import matplotlib.pyplot as plt
plot_df = df[df["revenue"] < 500]
plot_df = plot_df[plot_df["revenue"] > 0]
plt.scatter(plot_df["budget"], plot_df["revenue"])

In [ ]:
df.describe()

In [ ]:
df[["revenue", "budget"]].corr()